In [4]:
import torch
import torch.optim as optim
from torch.nn.functional import cross_entropy
import wandb

from src.RandmanFunctions import read_randman10_dataset
from src.Models import RandmanSNN
from src.EvolutionAlgorithms.EvolutionStrategy import ESModel
from src.Training import train_loop_snn

device = 'cpu'

In [7]:
print(torch.backends.cudnn.version())

None


In [5]:
# check if GPU is available
print(torch.cuda.is_available())

False


## Create Sweep

In [ ]:
sweep_config = {
    "method": "grid",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {        
        # Dataset:
        "nb_input": {"value": 100},
        "nb_output": {"value": 10},
        "nb_steps": {"value": 50},
        "nb_data_samples": {"value": 1000},
        # SNN:
        "nb_hidden": {"value": 10},
        "learn_beta": {"value": False},        
        # Training:
        "std": {"values": [0.05, 0.1]},
        "epochs": {"value":50},
        "batch_size": {"values": [64, 256]},
        # Optimization:
        "loss_fn": {"value": "cross-entropy"},
        "optimizer": {"value": "Adam"},
        "lr": {"value": 0.01},
        "regularization": {"value": "none"},
        # Evolution Strategy:
        "nb_model_samples": {"value": 20},
        "mirror": {"value": True},
    },
}

sweep_id = wandb.sweep(sweep_config, project="ES-Randman10")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 78qhh6zn
Sweep URL: https://wandb.ai/yixing/ES-Randman10/sweeps/78qhh6zn


## 🧹Sweep 

In [4]:
def train_snn(config=None):
    with torch.no_grad(), wandb.init(
        config=config
    ) as run:
        config = wandb.config
        # initialize Evolution Strategy instance
        es_model = ESModel(
            RandmanSNN,
            config.nb_input,
            config.nb_hidden,
            config.nb_output,
            0.95,
            sample_size=config.nb_model_samples,
            param_std=config.std,
            Optimizer=optim.Adam,
            lr=config.lr,
            device=device,
            mirror=config.mirror,
        )

        # load dataset
        train_loader, val_loader = read_randman10_dataset(
            "data/randman_10_dataset.pt", batch_size=config.batch_size
        )

        # epochs
        for epoch in range(config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")

            # train the model
            train_loop_snn(
            es_model, train_loader, val_loader, cross_entropy, device, run
            )

In [ ]:
# set sweep_id if picking up a sweep:
# sweep_id = ryhbaq55
wandb.agent(sweep_id, train_snn)

## Single Run (for old people)


In [ ]:
import wandb

device = "cpu"

def train_snn():
    run_name = "sanity_check_1"
    config = {  # Dataset:
        "nb_input": 100,
        "nb_output": 10,
        "nb_steps": 50,
        "nb_data_samples": 1000,
        # SNN:
        "nb_hidden": 10,
        "learn_beta": False,
        # Evolution Strategy:
        "nb_model_samples": 20,
        "mirror": True,
        # Training:
        "std": 0.05,
        "epochs": 50,
        "batch_size": 256,
        # Optimization:
        "loss": "cross-entropy",
        "optimizer": "Adam",
        "lr": 0.01,
        "regularization": "none",
    }
    with torch.no_grad(), wandb.init(
        entity="DarwinNeuron", project="ES-Randman10", name=run_name, config=config
    ) as run:
        # initialize Evolution Strategy instance
        es_model = ESModel(
            RandmanSNN,
            run.config.nb_input,
            run.config.nb_hidden,
            run.config.nb_output,
            0.95,
            sample_size=run.config.nb_model_samples,
            param_std=run.config.std,
            Optimizer=optim.Adam,
            lr=run.config.lr,
            device=device,
            mirror=run.config.mirror,
        )

        # load dataset
        train_loader, val_loader = read_randman10_dataset(
            "data/randman_10_dataset.pt", batch_size=run.config.batch_size
        )

        # epochs
        for epoch in range(run.config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")

            # train the model
            train_loop_snn(
                es_model, train_loader, val_loader, cross_entropy, device, run
            )


train_snn()